In [35]:
import numpy
import math
import random

In [352]:
environment = numpy.zeros((5, 5))
trans_prob = 0.7
environment[3, 4] = 10


In [353]:
alpha = 0.5
gamma = 0.9

In [315]:
def get_legal_actions(pos):
    legal_actions = list()
    new_pos = list()
    all_actions = [[1, 0], [0, 1], [-1, 0], [0, -1]]
    for i in all_actions:
        try:
            new_pos = numpy.add(pos, numpy.array(i))
            legal_actions.append(i)
            env = environment[new_pos[0]][new_pos[1]]
            if any(j < 0 for j in new_pos):
                legal_actions.pop()
        except IndexError:
            legal_actions.pop()
            
        
    return legal_actions

In [316]:
def take_action(pos, action):
    # Take action in the actual environment and return a reward and new position based on environment dynamics
    if numpy.random.rand(1) < 0.2:
        new_pos = numpy.add(pos, action)
    else:
        new_pos = numpy.add(pos, random.choice(get_legal_actions(pos)))
    reward = environment[new_pos[0]][new_pos[1]]
    return new_pos, reward

In [354]:
mb_internal_environment = numpy.zeros((5, 5))


In [209]:
def mb_internal_environment_value(pos):
    value = mb_internal_environment[pos[0], pos[1]]
    return value

In [210]:
mf_internal_environment

<function __main__.mf_internal_environment>

In [211]:
def mf_internal_environment_value(pos, action):
    return q_value

In [212]:
def update_mb_internal_environment(pos, next_pos, reward):
    next_value_list = list()
    for i in get_legal_actions(next_pos):
        next_value_list.append(mb_internal_environment[next_pos[0]][next_pos[1]])
    print next_value_list
    next_value = max(next_value_list)
    print pos[0]
    print "here", mb_internal_environment[pos[0], pos[1]]
    mb_internal_environment[pos[0], pos[1]] = (1-alpha) * mb_internal_environment_value(pos) + (alpha) * (reward + gamma*next_value)

In [219]:
update_mb_internal_environment(numpy.array([1, 4]), numpy.array([2, 4]), 0)

[3.375, 3.375, 3.375]
1
here 0.0


In [220]:
print mb_internal_environment

[[ 0.       0.       0.       0.       0.     ]
 [ 0.       0.       0.       0.       1.51875]
 [ 0.       0.       0.       0.       3.375  ]
 [ 0.       0.       0.       0.       7.5    ]
 [ 0.       0.       0.       0.       0.     ]]


In [281]:
def dls(pos, curr_depth, max_depth):
    value_list = list()
    if curr_depth == max_depth:
        return mb_internal_environment_value(pos)
    else:
        actions = get_legal_actions(pos)
        for i in actions:
            next_pos = numpy.add(numpy.array(pos), numpy.array(i))
            value = mb_internal_environment_value(pos) + gamma * dls(next_pos, curr_depth+1, max_depth)
            value_list.append(value)
        if value_list:
            average_value = numpy.mean(numpy.array(value_list))
    return average_value

In [282]:
print dls([3, 4], 0, 2)

10.8429375


In [284]:
def dls_action(pos, max_depth):
    # Do depth Limited Search till max_depth on the mb_internal_environment
    actions = get_legal_actions(pos)
    next_value_list = list()
    next_pos_list = list()
    for i in actions:
        next_pos = numpy.add(numpy.array(pos), numpy.array(i))
        next_value_list.append(dls(next_pos, 0, max_depth))
        next_pos_list.append(next_pos)
    action = actions[numpy.argmax(next_value_list)]
    return action

In [289]:
def q_action(pos):
    # Look for q values in mf_internal_environment for given position
    actions = get_legal_actions(pos)
    next_value_list = list()
    next_pos_list = list()
    for i in actions:
        next_pos = numpy.add(numpy.array(pos), numpy.array(i))
        next_value_list.append(mb_internal_environment[next_pos[0], next_pos[1]])
        next_pos_list.append(next_pos)
    action = actions[numpy.argmax(next_value_list)]
    return action

In [290]:
print q_action([2, 3])

[0, 1]


In [358]:
def dual(num_iters, steps_per_iter):
    pos = [0, 0]
    sum_reward = 0
    max_depth = 4
    for i in range(num_iters):
        for j in range(steps_per_iter):
            if j%(i+1) == 0:
                action = dls_action(pos, max_depth)
                next_pos, reward = take_action(pos, action)
                update_mb_internal_environment(pos, next_pos, reward)
                pos = next_pos
            else:
                action = q_action(pos)
                next_pos, reward = take_action(pos, action)
                update_mb_internal_environment(pos, next_pos, reward)
                pos = next_pos
            sum_reward += reward
    return sum_reward

In [359]:
sum_reward = dual(24, 10)

[0.0, 0.0, 0.0]
0
here 0.0
[0.0, 0.0, 0.0, 0.0]
1
here 0.0
[0.165448828125, 0.165448828125, 0.165448828125, 0.165448828125]
1
here 0.0
[2.7140049197708822, 2.7140049197708822, 2.7140049197708822, 2.7140049197708822]
2
here 0.165448828125
[10.007258675712983, 10.007258675712983, 10.007258675712983, 10.007258675712983]
2
here 2.71400491977
[8.060992547925105, 8.060992547925105, 8.060992547925105]
2
here 10.0072586757
[26.866390476618442, 26.866390476618442, 26.866390476618442]
2
here 8.06099254793
[21.120371988440851, 21.120371988440851, 21.120371988440851]
3
here 26.8663904766
[1.8860824304729398, 1.8860824304729398, 1.8860824304729398]
2
here 21.1203719884
[1.6619489293578762, 1.6619489293578762]
1
here 1.88608243047
[1.6909182334475141, 1.6909182334475141, 1.6909182334475141]
0
here 1.66194892936
[1.5918876697303195, 1.5918876697303195]
1
here 1.69091823345
[0.32164728142846771, 0.32164728142846771, 0.32164728142846771]
0
here 1.59188766973
[0.0, 0.0, 0.0, 0.0]
0
here 0.321647281428
[

In [360]:
print sum_reward

190.0
